### How are Classes Constructed?

When we write a class such as this:

In [1]:
import math

class Circle(object):
    def __init__(self, x, y, r):
        self.x = x
        self.y = y
        self.r = r
        
    def area(self):
        return math.pi * self.r ** 2

Remember that a class is an **instance** of the `type` class:

In [2]:
type(Circle)

type

And `type` is a class itself, so it is callable (with some arguments), and is used to create classes, instances of the `type` class.

There are four main steps involved with creating instances of a class:

1. The class body is extracted - think of it as just a lump of text that contains code.
2. The class dictionary (used for the **class** state) is created for the class namespace
3. The body (extracted in 1), is executed in the class namespace (created in 2), thereby populating the class dictionary (in this case with two symbols, `__init__` and `area`)
4. A new `type` **instance** is constructed using the name of the class, the base classes (remember Python supports multiple inheritance), and that dictionary.

Let's actually step through this process manually ourselves:

First we need to look at the `exec` built-in method:

Let's try it out with a simple example first:

In [3]:
namespace = {}

exec('''
a = 10
b = 20
''', globals(), namespace)

And now let's see what's in the `namespace` dictionary:

In [4]:
namespace

{'a': 10, 'b': 20}

As you can see, that dictionary was used as the local namespace when the code (in the string) was executed. Of course, the code can contain any valid Python code, including function definitions:

In [5]:
exec('''
def add(a, b):
    return a + b
    
def mul(a, b):
    return a * b
''', globals(), namespace)

In [6]:
namespace

{'a': 10,
 'b': 20,
 'add': <function __main__.add(a, b)>,
 'mul': <function __main__.mul(a, b)>}

And we can use those functions, since now they are actual function objects in the namespace (dictionary):

In [7]:
namespace['add'](10, 20)

30

Remember what I told you about the class body scope? Well, this is it! And you should now understand why functions defined in that scope do not actually know anything about what else is in that scope - those functions are created independently of the dictionary into which they are inserted.

So, this is how we are going to "run" the class **body** in the context of the class namespace dictionary.

We'll also need to create a new `type` instance, so let's see what the signature for the `type` constructor is:

In [8]:
help(type)

Help on class type in module builtins:

class type(object)
 |  type(object_or_name, bases, dict)
 |  type(object) -> the object's type
 |  type(name, bases, dict) -> a new type
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __dir__(self, /)
 |      Specialized __dir__ implementation for types.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __instancecheck__(self, instance, /)
 |      Check if an object is an instance.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __setattr__(self, name, value, /)
 |      Implement setattr(self, name, value).
 |  
 |  __sizeof__(self, /)
 |      Return memory consumption of the type object.
 |  
 |  __subclasscheck__(self, subclass, /)
 |     

The constructor variant we are interested in is the third one. That one requires three things:
1. the `name` of the class
2. an tuple containing the `bases` - the classes this class inherits from (can be empty, in which case it just inherits from `object`)
3. the class namespace `dict`

Remember when I said that classes were basically just dictionaries? As you can see here, apart from the `name` and `bases`, all the functionality of the class is stored in the namespace dictionary!!

So now, let's go ahead and create our `Circle` class using this approach:

In [9]:
class_name = 'Circle'

In [10]:
class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r

def area(self):
    return math.pi * self.r ** 2
"""

In [11]:
class_bases = ()  # defaults to object

In [12]:
class_dict = {}

In [13]:
exec(class_body, globals(), class_dict)

Now that we have executed that code in that namespace, that dictionary has some content:

In [14]:
class_dict

{'__init__': <function __main__.__init__(self, x, y, r)>,
 'area': <function __main__.area(self)>}

And we can now create the `Circle` class, or type, by creating a new instance of `type`:

In [15]:
Circle = type(class_name, class_bases, class_dict)

In [16]:
Circle

__main__.Circle

In [17]:
type(Circle)

type

In [18]:
Circle.__dict__

mappingproxy({'__init__': <function __main__.__init__(self, x, y, r)>,
              'area': <function __main__.area(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Circle' objects>,
              '__weakref__': <attribute '__weakref__' of 'Circle' objects>,
              '__doc__': None})

As you can see the `Circle` namespace dict contains our functions `__init__` and `area`.

And we now have a `Circle` class that we can use just like before:

In [19]:
c = Circle(0, 0, 1)

In [20]:
c.x, c.y, c.r

(0, 0, 1)

In [21]:
c.area()

3.141592653589793

So as you can see, we use the `type` class to construct new types (classes), basically creating instances of `type`.

This is why we refer to `type` as a **metaclass**. It is a class used to construct classes.

Also, make sure you understand that `type` is callable in two different ways - depending on what arguments are passed to `type()` it will do different things:

Creates a new `type` instance:

In [22]:
Circle = type(class_name, class_bases, class_dict)

Returns the `type` of an object:

In [23]:
type(Circle)

type